In [1]:
!pip install datasets

In [2]:
!pip install transformers

In [3]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
bert_model = AutoModel.from_pretrained("distilroberta-base")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
import numpy as np
import torch

In [5]:
from datasets import load_dataset
dataset = load_dataset("coastalcph/fairlex", "ecthr")

Generating train split:   0%|          | 0/9000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset fairlex downloaded and prepared to /root/.cache/huggingface/datasets/coastalcph___fairlex/ecthr/1.0.0/b755f714459ab788a8e3f9167fe7463f79981775296915d36ac10fc58ea93737. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
import os
import pandas as pd

base_path = "**location_of_parsed_data**"
files = sorted([base_path+'/'+f for f in os.listdir(base_path)])
case_files = []
for file in files:
  df = pd.read_csv(file)
  df = df.values
  cases = []
  for row in df:
    try: 
      if(row[0].index("\n(z0")!=0):
        content=row[0][row[0].index("\n(z0"):]
      else:
        content = row[0]
    except:
      continue
    cases.append(content)
  case_files.append(cases)

train_case_files = case_files[:]

base_path = "**location_of_parsed_validation_data**"
files = sorted([base_path+'/'+f for f in os.listdir(base_path)])
case_files = []
for file in files:
  df = pd.read_csv(file)
  df = df.values
  cases = []
  for row in df:
    try: 
      if(row[0].index("\n(z0")!=0):
        content=row[0][row[0].index("\n(z0"):]
      else:
        content = row[0]
    except:
      continue
    cases.append(content)
  case_files.append(cases)

val_case_files = case_files[:]

In [7]:
train_labels = dataset['train']['labels']
print(train_labels[:5], len(train_labels))

val_labels = dataset['validation']['labels']
print(val_labels[:5], len(val_labels))

[[4], [], [3], [6], []] 9000
[[], [1], [1, 3], [], []] 1000


In [8]:
label_vocab = []
for label_group in train_labels:
  label_vocab.extend(label_group)
  label_vocab = list(set(label_vocab))
labels_vocab = sorted(label_vocab)
print(label_vocab)
print(len(label_vocab))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
10


In [9]:
import numpy as np
vectored_y = []
for label_group in train_labels:
    temp = np.zeros(len(label_vocab))
    for l in label_group:
        temp[label_vocab.index(l)] =1
    vectored_y.append(temp)
vectored_y = np.stack(vectored_y)

val_vectored_y = []
for label_group in val_labels:
    temp = np.zeros(len(label_vocab))
    for l in label_group:
        temp[label_vocab.index(l)] =1
    val_vectored_y.append(temp)
val_vectored_y = np.stack(val_vectored_y)

In [10]:
vectored_y = torch.from_numpy(vectored_y).float()
val_vectored_y = torch.from_numpy(val_vectored_y).float()

In [11]:
val_vectored_y.shape, vectored_y.shape, len(val_case_files), len(train_case_files)

(torch.Size([1000, 10]), torch.Size([9000, 10]), 1000, 9000)

In [12]:
from tqdm import tqdm
device = "cuda"

In [13]:
class HierBERTClassifier(torch.nn.Module):
    def __init__(self, bert_model, num_classes=10):
        super().__init__()
        self.bert_model = bert_model
        self.num_classes = num_classes
        self.seg_encoder = torch.nn.Transformer(d_model=self.bert_model.config.hidden_size,
                                                nhead=self.bert_model.config.num_attention_heads,
                                                batch_first=True,
                                                dim_feedforward=self.bert_model.config.intermediate_size,
                                                activation=self.bert_model.config.hidden_act,
                                                dropout=self.bert_model.config.hidden_dropout_prob,
                                                layer_norm_eps=self.bert_model.config.layer_norm_eps,
                                                num_encoder_layers=2, num_decoder_layers=0).encoder
        self.dropout = torch.nn.Dropout(self.bert_model.config.hidden_dropout_prob)
        self.classifier = torch.nn.Linear(768, num_classes)
        
        
    def forward(self, input_ids, attention_mask):
        encoded_outputs = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)[0]
        encoded_outputs = encoded_outputs.contiguous().view(1, input_ids.size(0), 64, 768)
        encoded_outputs = encoded_outputs[:, :, 0]
        seg_encoder_outputs = self.seg_encoder(encoded_outputs)
        pooled_output, _ = torch.max(seg_encoder_outputs, 1)
        output = self.dropout(pooled_output)
        output = self.classifier(output)
        return output
        
        
model = HierBERTClassifier(bert_model)
model.load_state_dict(torch.load('**path_to_pretrained_model**'))
model.to(device)
num_classes=10

In [14]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0000025) #0.00003

for epoch in range(20):
    bar = tqdm(zip(train_case_files,vectored_y), total=len(vectored_y))
    running_loss, running_acc, running_prec, running_recall, running_f1 = 0, 0, 0, 0, 0
    model.train()
    for batch_idx, (case_x, y) in enumerate(bar):
        case_x = case_x[-32:]
        y = y.to(device)
        y = y.unsqueeze(0)
        inputs = tokenizer(case_x, return_tensors="pt", padding="max_length", truncation=True, max_length=128).to(device)
        out = model(inputs['input_ids'], inputs['attention_mask'])
        optimizer.zero_grad()
        loss = torch.nn.functional.binary_cross_entropy(torch.sigmoid(out), y)
        loss.backward()
        optimizer.step()
        y_ = (out>0).float()
        acc = torch.mean((y==y_).float())
        running_acc += acc.item()
        running_loss += loss.item()
        tp, fp, tn, fn = 0,0,0,0
        for i,j in zip(y[0], y_[0]):
            if i==1 and j==1:
                tp += 1
            elif i==0 and j==1:
                fp += 1
            elif i==1 and j==0:
                fn += 1
            else:
                tn += 1
        if tp+fp>0:
            prec = tp/(tp+fp)
        else:
            prec = 0.0
        if tp+fn>0:
            recall = tp/(tp+fn)
        else:
            recall = 0.0
        if prec+recall>0:
            f1 = 2*prec*recall/(prec+recall)
        else:
            f1 = 0.0
        running_prec += prec
        running_recall += recall
        running_f1 += f1
        bar.set_description(str({"loop_type":"train", "epoch": epoch+1, "loss": round(running_loss/(batch_idx+1), 3), "acc": round(running_acc/(batch_idx+1), 3), "f1-score": round(running_f1/(batch_idx+1), 3), "precision": round(running_prec/(batch_idx+1), 3), "recall": round(running_recall/(batch_idx+1), 3)}))
    bar.close()
    bar = tqdm(zip(val_case_files,val_vectored_y), total=len(val_vectored_y))
    running_loss, running_acc, running_prec, running_recall, running_f1 = 0, 0, 0, 0, 0
    model.eval()
    for batch_idx, (case_x, y) in enumerate(bar):
        case_x = case_x[-32:]
        y = y.to(device)
        y = y.unsqueeze(0)
        inputs = tokenizer(case_x, return_tensors="pt", padding="max_length", truncation=True, max_length=128).to(device)
        out = model(inputs['input_ids'], inputs['attention_mask'])
        loss = torch.nn.functional.binary_cross_entropy(torch.sigmoid(out), y)
        y_ = (out>0).float()
        acc = torch.mean((y==y_).float())
        running_acc += acc.item()
        running_loss += loss.item()
        tp, fp, tn, fn = 0,0,0,0
        for i,j in zip(y[0], y_[0]):
            if i==1 and j==1:
                tp += 1
            elif i==0 and j==1:
                fp += 1
            elif i==1 and j==0:
                fn += 1
            else:
                tn += 1
        if tp+fp>0:
            prec = tp/(tp+fp)
        else:
            prec = 0.0
        if tp+fn>0:
            recall = tp/(tp+fn)
        else:
            recall = 0.0
        if prec+recall>0:
            f1 = 2*prec*recall/(prec+recall)
        else:
            f1 = 0.0
        running_prec += prec
        running_recall += recall
        running_f1 += f1
        bar.set_description(str({"loop_type":"eval", "epoch": epoch+1, "loss": round(running_loss/(batch_idx+1), 3), "acc": round(running_acc/(batch_idx+1), 3), "f1-score": round(running_f1/(batch_idx+1), 3), "precision": round(running_prec/(batch_idx+1), 3), "recall": round(running_recall/(batch_idx+1), 3)}))
    bar.close()
    torch.save(model.state_dict(), "model_bert_epoch"+str(epoch+1)+".pt")

{'loop_type': 'train', 'epoch': 1, 'loss': 0.253, 'acc': 0.903, 'f1-score': 0.392, 'precision': 0.435, 'recall': 0.374}: 100%|██████████| 9000/9000 [19:00<00:00,  7.89it/s]
{'loop_type': 'eval', 'epoch': 1, 'loss': 0.254, 'acc': 0.899, 'f1-score': 0.163, 'precision': 0.177, 'recall': 0.158}: 100%|██████████| 1000/1000 [01:18<00:00, 12.71it/s]
{'loop_type': 'train', 'epoch': 2, 'loss': 0.21, 'acc': 0.919, 'f1-score': 0.466, 'precision': 0.491, 'recall': 0.467}: 100%|██████████| 9000/9000 [19:24<00:00,  7.73it/s]
{'loop_type': 'eval', 'epoch': 2, 'loss': 0.231, 'acc': 0.912, 'f1-score': 0.288, 'precision': 0.304, 'recall': 0.293}: 100%|██████████| 1000/1000 [01:21<00:00, 12.31it/s]
{'loop_type': 'train', 'epoch': 3, 'loss': 0.191, 'acc': 0.927, 'f1-score': 0.513, 'precision': 0.537, 'recall': 0.516}: 100%|██████████| 9000/9000 [19:47<00:00,  7.58it/s]
{'loop_type': 'eval', 'epoch': 3, 'loss': 0.222, 'acc': 0.914, 'f1-score': 0.333, 'precision': 0.352, 'recall': 0.339}: 100%|██████████| 1

In [15]:
torch.save(model.state_dict(), "model_distilbert_pretrained_final.pt")